In [0]:

from bdce.common.utils import update_application_code
update_application_code(spark, role="BDP-PROD-APP-INT-QA", application_name="zidong-application-autopipeline")

# reload dependencies from temp
spark.sparkContext.addPyFile("/tmp/zeppelin_application_code/libs/python/dependencies.zip")
# spark.sparkContext.addPyFile("/home/hadoop/bdp/application/libs/python/dependencies.zip")
import aaplproxy

In [0]:

from pyspark.sql.functions import sum
from pyspark.sql.functions import desc
from aadatapipelinecore.core.urn import Urn
import psycopg2
from pyspark.sql import Row
from dateutil.relativedelta import relativedelta
from aaplproxy.connection import ClusterConnection
from conf import settings
from aadatapipelinecore.core.loader.plproxy import build_db_settings
import datetime as d
import datetime
from datetime import timedelta

test_result = []
PG_AA_HOSTS = [('10.2.6.141', 5432)]
PG_AA_NAME = 'aa_store_db'
PG_AA_ACCESS_ID = 'citus_bdp_prod_app_int_qa'
PG_AA_SECRET_KEY = 'wZw8cfBuuklIskVG'

aa_dsn = (
    "dbname='{db}' user='{user}' password='{password}' "
    "host='{host}' port='{port}'".format(
        db=PG_AA_NAME,
        user=PG_AA_ACCESS_ID,
        host=PG_AA_HOSTS[0][0],
        password=PG_AA_SECRET_KEY,
        port=PG_AA_HOSTS[0][1]
    )
)

urn = Urn(
    namespace="app-qa.db-check.v1",
    owner="app_qa"
)
sql = """select distinct date from store.store_est_t_{}_fact_v1 where granularity='{}';"""


def query(dsn, sql):
    with psycopg2.connect(dsn) as conn:
        conn.autocommit = True
        with conn.cursor() as cur:
            cur.execute(sql)
            result = cur.fetchall()
            conn.commit()
    return result


dict_test = {'weekly': ['2010-07-10', '2020-06-20'],
             'monthly': ['2010-07-31', '2020-04-30'],
             'quarterly': ['2010-09-30', '2020-03-31'],
             'yearly': ['2010-12-31', '2019-12-31']
             }


def last_day_of_month(check_month):
    next_month = check_month.replace(day=28) + datetime.timedelta(days=4)
    return next_month - datetime.timedelta(days=next_month.day)
    
    
def get_date_range(date_list, granularity):
    result = []
    start = datetime.datetime.strptime(date_list[0], '%Y-%m-%d')
    end = datetime.datetime.strptime(date_list[1], '%Y-%m-%d')
    if granularity == 'weekly':
        while start <= end:
            date_row = datetime.datetime.strftime(start, '%Y-%m-%d')
            result.append(date_row)
            start += relativedelta(weeks=1)
    elif granularity == 'monthly':
        while start <= end:
            start = last_day_of_month(start)
            date_row = datetime.datetime.strftime(start, '%Y-%m-%d')
            result.append(date_row)
            start += relativedelta(months=1)
    elif granularity == 'quarterly':
        while start <= end:
            start = last_day_of_month(start)
            date_row = datetime.datetime.strftime(start, '%Y-%m-%d')
            result.append(date_row)
            start += relativedelta(months=3)
    elif granularity == 'yearly':
        while start <= end:
            date_row = datetime.datetime.strftime(start, '%Y-%m-%d')
            result.append(date_row)
            start += relativedelta(months=12)
    # print result
    return result


def check_store_db_illegal_date(date_list, graularity):
    date_list = set(date_list)
    db_date_tuple_list = query(aa_dsn, sql.format(graularity[0], graularity))
    db_date_list = [datetime.datetime.strftime(x[0], '%Y-%m-%d') for x in db_date_tuple_list]
    db_date_list = set(db_date_list)
    print db_date_list.difference(date_list)


for key, value in dict_test.items():
    print key, value
    check_store_db_illegal_date(get_date_range(value, key), key)
    

In [0]:

from pyspark.sql.functions import sum
from pyspark.sql.functions import desc
from aadatapipelinecore.core.urn import Urn
import psycopg2
from pyspark.sql import Row
from dateutil.relativedelta import relativedelta
from aaplproxy.connection import ClusterConnection
from conf import settings
from aadatapipelinecore.core.loader.plproxy import build_db_settings
import datetime as d
import datetime
from datetime import timedelta

test_result = []
PG_AA_HOSTS = [('10.2.6.141', 5432)]
PG_AA_NAME = 'aa_store_db'
PG_AA_ACCESS_ID = 'citus_bdp_prod_app_int_qa'
PG_AA_SECRET_KEY = 'wZw8cfBuuklIskVG'

aa_dsn = (
    "dbname='{db}' user='{user}' password='{password}' "
    "host='{host}' port='{port}'".format(
        db=PG_AA_NAME,
        user=PG_AA_ACCESS_ID,
        host=PG_AA_HOSTS[0][0],
        password=PG_AA_SECRET_KEY,
        port=PG_AA_HOSTS[0][1]
    )
)

urn = Urn(
    namespace="app-qa.db-check.v1",
    owner="app_qa"
)
sql = """select distinct date from store.store_est_category_t_{}_fact_v1 where granularity='{}';"""


def query(dsn, sql):
    with psycopg2.connect(dsn) as conn:
        conn.autocommit = True
        with conn.cursor() as cur:
            cur.execute(sql)
            result = cur.fetchall()
            conn.commit()
    return result


dict_test = {'weekly': ['2010-07-10', '2020-06-20'],
             'monthly': ['2010-07-31', '2020-04-30'],
             'quarterly': ['2010-09-30', '2020-03-31'],
             'yearly': ['2010-12-31', '2019-12-31']
             }


def last_day_of_month(check_month):
    next_month = check_month.replace(day=28) + datetime.timedelta(days=4)
    return next_month - datetime.timedelta(days=next_month.day)
    
    
def get_date_range(date_list, granularity):
    result = []
    start = datetime.datetime.strptime(date_list[0], '%Y-%m-%d')
    end = datetime.datetime.strptime(date_list[1], '%Y-%m-%d')
    if granularity == 'weekly':
        while start <= end:
            date_row = datetime.datetime.strftime(start, '%Y-%m-%d')
            result.append(date_row)
            start += relativedelta(weeks=1)
    elif granularity == 'monthly':
        while start <= end:
            start = last_day_of_month(start)
            date_row = datetime.datetime.strftime(start, '%Y-%m-%d')
            result.append(date_row)
            start += relativedelta(months=1)
    elif granularity == 'quarterly':
        while start <= end:
            start = last_day_of_month(start)
            date_row = datetime.datetime.strftime(start, '%Y-%m-%d')
            result.append(date_row)
            start += relativedelta(months=3)
    elif granularity == 'yearly':
        while start <= end:
            date_row = datetime.datetime.strftime(start, '%Y-%m-%d')
            result.append(date_row)
            start += relativedelta(months=12)
    # print result
    return result


def check_store_db_illegal_date(date_list, graularity):
    date_list = set(date_list)
    db_date_tuple_list = query(aa_dsn, sql.format(graularity[0], graularity))
    db_date_list = [datetime.datetime.strftime(x[0], '%Y-%m-%d') for x in db_date_tuple_list]
    db_date_list = set(db_date_list)
    print db_date_list.difference(date_list)


for key, value in dict_test.items():
    print key, value
    check_store_db_illegal_date(get_date_range(value, key), key)
    

In [0]:

end = '2012-09-30'
granularity = 'monthly'
pre_agg_df = spark.read.format("delta").load("s3://b2c-prod-data-pipeline-unified-store-paid/unified/" \
            "store.app-est-category.v3/fact/").where("granularity='{}' and date='{}' and data_stage='final'".format(granularity, end))
pre_agg_df.show()

In [0]:

end = '2015-04-25'
granularity = 'weekly'
pre_agg_df = spark.read.format("delta").load("s3://b2c-prod-data-pipeline-unified-store-paid/unified/" \
            "store.app-est-category.v3/fact/").where("granularity='{}' and date='{}' and data_stage='final'".format(granularity, end))
print pre_agg_df.count()

In [0]:

from pyspark.sql.functions import sum
from pyspark.sql.functions import desc

start = '2012-01-01'
end = '2012-12-31'
granularity = 'yearly'

category_daily_df = spark.read.format("delta").load("s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est-category-load.v3/fact/").where("granularity='daily' and date between '{}' and '{}'".format(start, end))


agg_df = category_daily_df.groupBy('app_id', 'country_code', 'device_code', 'category_id').agg(
    sum('est_free_app_download').alias('free_app_download'), sum('est_paid_app_download').alias('paid_app_download'), 
        sum('est_revenue').alias('revenue'))
agg_df.createOrReplaceTempView("agg_df")

if granularity == 'weekly' or granularity == 'monthly':
    pre_agg_df = spark.read.format("delta").load("s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est-category.v3/fact/").where("granularity='{}' and date='{}' and data_stage='final'".format(granularity, end))
else:
    pre_agg_df = spark.read.format("delta").load("s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est-category-pre-aggr.v3/fact/").where("granularity='{}' and date='{}' and data_stage='final'".format(granularity, end))

pre_agg_df.createOrReplaceTempView("pre_agg_df")

diff_df1 = spark.sql("""select app_id, category_id, country_code, device_code, free_app_download, paid_app_download, revenue 
                            from agg_df 
                        except all 
                        select app_id, category_id, country_code, device_code, free_app_download, paid_app_download, revenue 
                            from pre_agg_df""")
diff_df2 = spark.sql("""select app_id, category_id, country_code, device_code, free_app_download, paid_app_download, revenue 
                            from pre_agg_df 
                        except all 
                        select app_id, category_id, country_code, device_code, free_app_download, paid_app_download, revenue 
                            from agg_df""")
print agg_df.count(), pre_agg_df.count()
print diff_df1.count(), diff_df2.count()
diff_df1.show()
diff_df2.show()